# Overview

Recall the definition of $P_{ST}^{\sigma_{\Gamma}}$ in Section $2.4$ of the thesis.

In this notebook we consider the graphs of Section $1.3$ of the thesis, and for a given graph $\Gamma$, determine whether the convex region $P_{ST}^{\sigma_{\Gamma}}$ is empty or not for all $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ (obtained via 4\_assignments.ipynb and 5_assignments\_11k3graphs.ipynb), and therefore conclude the existence/non-existence of a numerical polarisation for each $\sigma_{\Gamma}$.

As $P_{ST}^{\sigma_{\Gamma}}$ is a convex region if $P_{ST}^{\sigma_{\Gamma}}$ has dimension $|V(\Gamma)|-1$ then $P_{ST}^{\sigma_{\Gamma}} \ne \emptyset$ and if $P_{ST}^{\sigma_{\Gamma}}$ has dimension strictly less than $|V(\Gamma)|-1$ then $P_{ST}^{\sigma_{\Gamma}} = \emptyset$.

Our method to show $P_{ST}^{\sigma_{\Gamma}} \ne \emptyset$ is to modify the definition of $P_{ST}^{\sigma_{\Gamma}}$ and swap $<$ with $\le$. This modification allows us to utilize the Sage function `Polyhedron()` to construct $P_{ST}^{\sigma_{\Gamma}}$ and determine its dimension. We can then ascertain whether, with the modified inequality ($\le$), the dimension of $P_{ST}^{\sigma_{\Gamma}}$ is strictly less than $|V(\Gamma)|-1$, which implies that $P_{ST}^{\sigma_{\Gamma}}$ is indeed an empty set.

### Results

We see that all graphs consider in the Section $1.3$, with the expection of $F$ given by Filippo Viviani have $P_{ST}^{\sigma_{\Gamma}} \ne \emptyset$ for all $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$. For the graph $F$ there are $16$ stability conditions $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ such that $P_{ST}^{\sigma_{\Gamma}} = \emptyset$. It remains to show that these $16$ are in fact strong stability conditions, we do this in "7_strong_stability_check.ipynb".

### Notes


1) For each of the graphs in #Cases we load $\Sigma_{\Gamma,T,D_{T}}^{d}$ from "\Stability_Conditions\Graph\phi_investigation\examples\{graphname}".

2) We store the results as a list in "\Stability_Conditions\Graph\phi_investigation\examples\{graphname}\phi_analysis"

# Table of contents

1. [Functions](#s1)
    1. [Main Functions](#s11)
    2. [Secondary Functions](#s12)
    3. [Loading and storing](#s13)
2. [Example](#s3)    
3. [Cases](#s2)
    1. [$n$-necklace graphs $n \le 5$](#s21)
    2. [$\Gamma:=\Gamma_{1 1 k_3}$ for $2 \le k_3 \le 6$](#s22)
    3. [$\Gamma:=\Gamma_{1 k_2 k_3}$ for $k_2\ge 2$](#s23)
    4. [$\Gamma:=\Gamma_{k_1 k_2 k_3}$ for $k_1\ge 2$](#s24)
    5. [$\Gamma \in \{ G_1,G_2,G_3,F\}$](#s25)

# Functions <a name="s1"></a>

## Used throughout


In [ ]:
import numpy as np
import itertools
import pickle
from fractions import Fraction
from collections import Counter
import textwrap

## Main functions <a name="s11"></a>

We now determine for each $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ the dimension of $P_{ST}^{\sigma_{\Gamma}}$. The inputs for which are the graph $G$ and the set $\Sigma_{\Gamma,T,D_{T}}^{d}$ as data.

In [ ]:
def analyse_graphname(G,data,start=0,terminator=False): #general not FV example
    """
    Obj: Measure for each stability condition the dimension of P_{ST}^{\sigma_{\Gamma}}
    #inputs:   
    
    size: number of vertices of graph minus 1.
    graphname:name in folder to load the data
    G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
    #     start: the ith term 1,...,k you want to start the computation from.
    #     terminator = is a number 1,...,k, the nth to which we calculate upo to so 
    not to have to calc all cases (used for examples).
    
    Returns: a list of dictionaries of the form {"Dimension of tree-poly":dim_tree_poly}
    where tree-poly is the region P_{ST}^{\sigma_{\Gamma}} for each stability codnition
    and dim_tree_poly is its dimension.
    """
    
    dic_l=[]
    top_dim=[] # record those with top dim poly
    degen=[] #record wsc with degen poly.
    
    for term,wsc in enumerate(data):
        
        #Use start to skip to the case you want to study.
        if term<start-1:
            continue
        
        print("Start=",start,"----",term,"----terminator:",terminator)
        
        #for Tree_poly
        assignments=wsc[2] #[(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0])]
        tree_poly=get_intersection_tree_poly(G,assignments)
        dic={"Dimension of tree-poly":dim(tree_poly)}
        dic_l.append(dic)

    
    return dic_l

Recall the definition of $P_{ST}^{\sigma_{\Gamma}}$ as $P_{ST}^{\sigma_{\Gamma}}= \bigcap_{T \in \mathcal{ST}(\Gamma)}P_{T}^{\sigma_{\Gamma}}$, given in Section $2.4$ of the thesis. The main subfunction of analyse\_graphname is get\_intersection\_tree\_poly which gives the region $P_{ST}^{\sigma_{\Gamma}}$ 

In [ ]:
def get_intersection_tree_poly(G,assignments):

    """
    Objective: get P_{ST}^{\sigma_{\Gamma}}
    Input: 
    assignments: the tree assignment A_{\Gamma} for \sigma_{\Gamma} as a list of pairs (Tree,divisor).
    Returns: P_{ST}^{\sigma_{\Gamma}} as a polyhedron (with \le)
    """
    
    #Build and record polytopes.
    poly_l=[]
    for ass in assignments:
    
        T,divisor=ass[0],ass[1]
    
        P=get_tree_poly(G,T,divisor)
        
        poly_l.append(P)

    #Take intersection of polytopes for trees
    inter=poly_l[0]
    for P in poly_l[1:]:
        inter=inter.intersection(P)

    return inter

Let $\sigma_\Gamma \in \Sigma_{\Gamma}^{d}$. To construct $P_{ST}^{\sigma_{\Gamma}}$, it is necessary to take the intersection of $P_{T}^{\sigma_{\Gamma}}$ for all $T \in \mathcal{\Gamma}$. The following function get\_tree\_poly gives $P_{T}^{\sigma_{\Gamma}}$ for a $T \in \mathcal{ST}(\Gamma)$ and $\vec{d} \in \sigma_\Gamma(T)$.

In [ ]:
def get_tree_poly(G,T,divisor):#Main
    
    """
    Objective:Get P_{T}^{\sigma_{\Gamma}} for a tree using the phi-tree-inequalties.
    Input: 
    divisor: d given by \sigma_{\Gamma}(T)
    T: Spanning tree of \Gamma
    G is graph
    Returns: Polyhedron P_{T}^{\sigma_{\Gamma}} (with \le)
    """

    # get list of phi_tree_inequalities_data(G,T) for each trees.(dictionary)
    phi_G=phi_tree_inequalities_data(G,T)

    hyperp=phi_tree_formatted_hyperplanes(divisor,phi_G,G)

    #build a polytope.
    tree_poly=Polyhedron(ieqs=hyperp)

    return tree_poly

To state $P_{T}^{\sigma_{\Gamma}}$ it is necessary to have the hyperplanes which give $P_{T}^{\sigma_{\Gamma}}$.

In [ ]:
def phi_tree_formatted_hyperplanes(divisor,phi_G,G):
    
    """
    Objective: Get list of inquality data to feed into Sages Polyhedron function
    https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/polyhedron/constructor.html
    
    Polyhedron(ieqs=[(0,1,0),(0,0,1),(1,-1,-1)]).Hrepresentation()
    (An inequality (-1, -1) x + 1 >= 0,
     An inequality (1, 0) x + 0 >= 0,
     An inequality (0, 1) x + 0 >= 0)
     
    to add the total degree inequality x+y+z..=degree.
    
    Ax=b
    
    Further Notes:    
    Here divisor is a single divisor. Not for all divisors in a set.
    
    Inputs:
    phi_G =  A dictionary that records indexs (vertices of \Gamma^{'}) of cuts (\Gamma^{'},\Gamma^{'c}) as keys, 
    and value=[num_e_in_tree,num_e_notin_tree]
    divisor= divisor in stability condition for tree ie. sigma_{\Gamma}(T)
    
    Returns: list of inequalities in the correct format to pass to Polyhedron as above.

    """
    
    #length of divisor:
    n=len(divisor)
    
    # We add the total degree equation to inequals for phi
    deg=get_first_betti_number(G) #will be 0 for trees.
    tot_equ1=[-deg,]+ [1]*n
    tot_equ2=[deg,]+ [-1]*n
    
    #List of inequalities we wish to return
    inequals=[tot_equ1,tot_equ2]     
        
    for indexor in list(phi_G.keys()):

        num_e_in_tree,num_e_notin_tree,gamma0_missing_edges=phi_G[indexor] #value=[num_e_in_tree,num_e_notin_tree]    

        res_upper=upper_part(indexor,n,divisor,num_e_in_tree,num_e_notin_tree,gamma0_missing_edges)

        res_lower=lower_part(indexor,n,divisor,num_e_in_tree,num_e_notin_tree,gamma0_missing_edges)

        res_combined= [res_lower,res_upper]

        inequals=inequals+res_combined
    return inequals

In phi\_tree\_formatted\_hyperplanes the main subfunctions of which are:

1) get\_first\_betti\_number
2) lower\_part and upper\_part.

The data of phi\_G is given by phi\_tree\_inequalities\_data (which stores the cut information for all induced elemenart subgraphs).

We state how we determine 1). We take $d=b_{1}(\Gamma)$ to be the first betti number of the graph.
As a consequence we have that for $\vec{d} \in \sigma_{\Gamma}(T)$ we have $deg(\vec{d})=0$ for any spanning tree $T$.

In [ ]:
def get_first_betti_number(G):
    #for if the degree of divisors on trees is 0, the degree of lbm is this.
    return len(G.edges())-len(G.vertices())+1

We state how we determine 2).

For $\vec{d} \in \sigma_{\Gamma}(\Gamma_0)$ for $\Gamma_0$ any connected spanning subgraph or $\phi \in V^{d}(\Gamma)$, the following function allow one to determine $d_{\Gamma^{'}}= \sum_{v \in V\left(\Gamma^{\prime}\right)}\vec{d}(v)$ and $\phi_{\Gamma^{'}}= \sum_{v \in V\left(\Gamma^{\prime}\right)}\phi(v)$ for $\Gamma^{'}$ an induced elementary subgraph.



In [ ]:
def sum_comps(comp,indexor): #Done
    """returns the sum of the components of comp in the position by indexor"""
    #for d and phi
    sum_l=[comp[int(i)] for i in indexor] #Becuase of the vertex labeling of G having v_0....
    return sum(sum_l)

Recall for each  elementary induced subgraph $\Gamma^{'}$ defines two hyperplanes that give the bounds for $\phi_{\Gamma^{'}}$ that is is $L_{\Gamma_0,\Gamma^{'}}+ \vec{d}_{\Gamma^{'}}< \phi_{\Gamma^{'}}  <U_{\Gamma_0,\Gamma^{'}}+ \vec{d}_{\Gamma^{'}}$ (given by lower\_part and upper\_part).

Let $\vec{d} \in \sigma_{\Gamma}(\Gamma_0)$ and consider the pair of inequalities,

$$L_{\Gamma_0,\Gamma^{'}}< \phi_{\Gamma^{'}} - \vec{d}_{\Gamma^{'}} <U_{\Gamma_0,\Gamma^{'}}$$

The function lower\_part allows one to calculate 

$$\phi_{\Gamma^{'}} <U_{\Gamma_0,\Gamma^{'}} + \vec{d}_{\Gamma^{'}} .$$


and the function upper\_part allows one to calculate ($L_{\Gamma_0,\Gamma^{'}}+ \vec{d}_{\Gamma^{'}}< \phi_{\Gamma^{'}}$) but we write it in the following method so we can take the inequality in the same direction,

$$-\phi_{\Gamma^{'}}<-L_{\Gamma_0,\Gamma^{'}}- \vec{d}_{\Gamma^{'}}$$

For the functions lower part and upper part the following terms correspond to:

1) T\_term:
    
$$\frac{\left|\operatorname{E}\left(\Gamma_0\right) \backslash\left(\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right)\right)\right|}{2}$$
    

2) T\_comp\_term:
    
$$\frac{\left|\operatorname{E}\left(\Gamma_0^c\right) \backslash (\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right))\right|}{2}$$
    
3) divisor\_comp\_sum:
    
$$d_{\Gamma^{'}}= \sum_{v \in V\left(\Gamma^{\prime}\right)}\vec{d}(v)$$
    
4) totaldeg\_divisor\_comp\_sum:
    
$$d_{\Gamma^{'}}+\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|$$
    
    

5) bterm:
    
    i) for lower\_part is

$$d_{\Gamma^{'}}+\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|+\frac{\left|\operatorname{E}\left(\Gamma_0\right) \backslash\left(\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right)\right)\right|}{2} +\frac{\left|\operatorname{E}\left(\Gamma_0^c\right) \backslash (\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right))\right|}{2}$$     

5) bterm:
    
    i) for upper\_part is

$$-d_{\Gamma^{'}}-\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|+\frac{\left|\operatorname{E}\left(\Gamma_0\right) \backslash\left(\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right)\right)\right|}{2} -\frac{\left|\operatorname{E}\left(\Gamma_0^c\right) \backslash (\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right))\right|}{2}$$     

In [ ]:
def lower_part(indexor,n,divisor,num_e_in_tree,num_e_notin_tree,gamma0_missing_edges):

    #get b term
    T_term=num_e_in_tree*0.5
    
    T_comp_term=num_e_notin_tree*0.5
    
    divisor_comp_sum=sum_comps(divisor,indexor)
    
    totaldeg_divisor_comp_sum=divisor_comp_sum+gamma0_missing_edges#Corrected to total degree
    
    bterm=[totaldeg_divisor_comp_sum+T_term+T_comp_term] #See phi_tree_inequality_doc
    
    base=np.zeros(n)
    
    """
    It is very important whether its (1,i) or (-1,i) below in l
    """
    l=[(-1,i) for i in indexor] # put -1 in the ith position given be indexor. 

    for item in l:
        ipos = int(item[1]) # ith position
        val=item[0]
        base[ipos]=val

    xyz=base

    ineq=bterm+list(xyz) # [b,x,y,z...] want to concat                
    return ineq

def upper_part(indexor,n,divisor,num_e_in_tree,num_e_notin_tree,gamma0_missing_edges):

    #Remember upper part of modulus inequality.

    #get b term
    T_term=num_e_in_tree*0.5
    T_comp_term=num_e_notin_tree*0.5
    divisor_comp_sum=sum_comps(divisor,indexor)
    totaldeg_divisor_comp_sum=divisor_comp_sum+gamma0_missing_edges#Corrected to total degree

    bterm=[-totaldeg_divisor_comp_sum+T_term-T_comp_term] #See phi_tree_inequality_doc

    #get x,y,z term
    base=np.zeros(n)
    l=[(1,i) for i in indexor] # put -1 in the ith position given be indexor.

    for item in l:
        ipos = int(item[1]) # ith position
        val=item[0]
        base[ipos]=val

    xyz=base

    ineq=bterm+list(xyz) # [b,x,y,z...] want to concat

    return ineq

We now state the information required for upper\_part and upper\_part.

Let $\Gamma^{'}$ be a induced elementary subgraph of $\Gamma$, and $\Gamma_0=T \in \mathcal{ST}(\Gamma)$ (also works for connected spanning subgraphs).

The data of phi\_G is given by the following function phi\_tree\_inequalities\_data stores the cut information for all induced elementary subgraphs. In particular it returns a dictionary of keys of vertices of each induced elementary subgraph of $\Gamma$, with values being a list of the 

1) the number of edges in $\Gamma^{'}$ and $\Gamma_0$

2) the number of edges not in $\Gamma^{'}$ and $\Gamma_0$

3) the number of edges $\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|$.

In [ ]:
def phi_tree_inequalities_data(G,T): #Main
    
    """
    Objective: Used in tree_poly
    Inputs: 
    G: graph
    T:graph of spanning tree (or any spanning connected subgraph)
    
    Return:dictionary of data used to define the phi-tree inequalties. 
    Returns phi_G

    Dict contains the vertices of the cut : key
    List of [e_in_tree ,e_notin_tree]: value
    
    Where num_e_in_tree = edges of cut the belong to tree
         ,num_e_notin_tree= edges of cut that do not belong to tree
         
    terms=[num_e_in_tree ,num_e_notin_tree,gamma0_missing_edges]

    """
    
    #for main

    pairs=get_pairs_con_subg(G)

    inequ_data=[]
    for pair in pairs: 
        
        #As only care about one complete subg, we get the number of missing edges of T and complete_subg.
        complete_subg=pair[0]
        gamma0_missing_edges=get_gamma0_missing_edges(G,T,complete_subg)
        
        
        #Would like to know the edges that G has in the cut. i.e. cut_edges_G
        cut_edges=get_cut_edges(G,pair)

        num_e_in_tree ,num_e_notin_tree=edges_cut_inandnotin_tree(cut_edges,T)

        terms=[num_e_in_tree ,num_e_notin_tree,gamma0_missing_edges]
        
#         print("pair[0].vertices():",pair[0].vertices(),"pair[1].vertices():",pair[1].vertices())
#         print("[num_e_in_tree ,num_e_notin_tree]",terms)


        "Do we need to add the data for both subg in pair?"

        #get vertices of subg in pair and turn to "v1v2v3" format.
        subg1_l=pair[0].vertices()
        subg1 = ''.join(subg1_l)
        inequ_data.append((subg1,terms)) #subg1 is the indexor in phi_tree_formatted_hyperplanes

#         print("(subg1,terms):",(subg1,terms))
#         subg2_l=pair[1].vertices()
#         subg2 = ''.join(subg2_l)
#         inequ_data.append((subg2,terms))


    #Now join to phi-inequal data dict
    phi_tree_iequal_dict = {t[0]: t[1] for t in inequ_data}

    return phi_tree_iequal_dict

We first need to determine the set of all elementary induced subgraphs $(\Gamma^{'},\Gamma^{'c})$.

In [ ]:
def get_pairs_con_subg(G):# for phi_tree_inequalities_data
    
    """
    Note: we are not working with multigraphs so connected_subgraph_iterator should give subgraphs with all edges between    
    Objective: Used to get cut info and total degree adjustment
    Input:
    Return: Pairs of complete subgraphs for V and V^{c} such that V scup V^{c}=V(G)
    """
    
    vert_G=G.vertices()
    edges_G=G.edges()
    
    #get list of connected subgraphs:
    
    #Want to get pairs of complete subgraphs for pairs of vertices that is a is disjoint union of all verts
    ll=list(G.connected_subgraph_iterator())

    # put them into pairs, where vertices are a disjoint union of G.
    pairs=[]
    for i in ll:
        a=i.vertices()
        for j in ll:
            b=j.vertices()
            if sorted(a+b)==vert_G:
                #May need to take complete graph here containing a, and for b. 
                pairs.append((i,j))
                
#                 """
#                 We only need one subgraph in the cut, the following step prevents us having both subg in a cut.
#                 """
#                 if (i,j) in pairs:
#                     continue
#                 if (j,i) in pairs:
#                     continue
#                 else:
#                     pairs.append((i,j))
                    
#     for i in get_pairs_con_subg(G):
#         print(i[0].vertices(),i[1].vertices())
    return pairs

Let $\Gamma_0 :=T$. The next functions allow us to determine:

 the number of edges in a given cut $(\Gamma^{'},\Gamma^{'c})$ belonging to the tree and in particular

   $$\frac{\left|\operatorname{E}\left(\Gamma_0^c\right) \backslash (\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right))\right|}{2}$$
   
  the number of edges in a given cut $(\Gamma^{'},\Gamma^{'c})$ belonging to $\Gamma \setminus \Gamma_0$ and in particular
  
  $$\frac{\left|\operatorname{E}\left(\Gamma_0\right) \backslash\left(\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right)\right)\right|}{2}.$$


  

In [ ]:
def get_cut_edges(G,pair): #for phi_tree_inequalities_data
    
    """
    Obj:Get list of the edges in a cut for a given pair, 
    for a pair of connected subgraphs that partitions the vertices of G.
    
    Return: list of edges in cut for a pair of connected subgraphs that partition the vertices of G 
    """
    a_edges=pair[0].edges()
    b_edges=pair[1].edges()
    total_edges=a_edges+b_edges
    
    edges_G=G.edges()

    #Cut edges.
    cut_edges=[ e for e in edges_G if e not in total_edges]
    
    return cut_edges

def edges_cut_inandnotin_tree(cut_edges,tree):#Used in phi_tree_inequalities_data
    
    """
    
    Obj: Consider cut_edges_G and ask if any of the edges of T are in cut_edges_G

    Returns: Number of edges in tree, and not in tree
    """

    tree_edges=tree.edges()

    e_in_tree=[edge for edge in cut_edges if edge in tree_edges]
    # print("edges of cut that belong to tree:",tree_edges_cut)

    e_notin_tree=[edge for edge in cut_edges if edge not in tree_edges]
    # print("edges of cut that do not belong to tree:",tree_comp_edges_cut)
    
    num_e_in_tree ,num_e_notin_tree=len(e_in_tree),len(e_notin_tree)
    
    return num_e_in_tree ,num_e_notin_tree

Finally the last function llows us to calculate the number of edges $\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|$.

In [ ]:
def get_gamma0_missing_edges(G,T,complete_subg):
    """
    Objective:find adjustment to degree of divisors, a number, to correct to the total degree.
    given by the number of edges in complete subgraph which are not in gamm0
    Input: gamma0 (the tree), and the complete_subg
    Return: number of edges complete_subg has over T (on one side of the  cut).
    
    
    Want |E(gamma0^c) cap E(gamma')| where gamma' is the complete subgraph and gamma0 the connected spanning subg.
    """
    
    
    gamma0_complement=[e for e in G.edges() if e not in T.edges()]
    
    gamma0_comp_inter_complete_subg=[e for e in gamma0_complement if e in complete_subg.edges()]
    
    return len(gamma0_comp_inter_complete_subg)

## Secondary Functions <a name="s12"></a>

As the graph $F$ is known to have $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ where the dimension of $P_{ST}^{\sigma_{\Gamma}}$ is strictly less than $|V(\Gamma)|-1$, we therefore use following function, which is analogous to analyse\_graphname, where we record those cases where $P_{ST}^{\sigma_{\Gamma}}$ the dimension is less than $|V(\Gamma)|-1$, and those that have dimension $|V(\Gamma)|-1$.

In [ ]:
def analyse_graphname_FV(size,graphname,G,data,start=0,terminator=False): 
    """
    Obj: Create a table of results for the graph F: 
    As it has fail cases when dim_tree_poly< |V(\Gamma)|-1

    #inputs:    
    size: number of vertices of graph minus 1.
    graphname:name in folder to load the data
    G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
    #     start: the ith term 1,...,k you want to start the computation from.
    #     terminator = is a number 1,...,k, the nth to which we calculate upo to so 
    not to have to calc all cases (used for examples).
    
    Returns: a list of dictionaries of the form {"Dimension of tree-poly":dim_tree_poly}
    where tree-poly is the region P_{ST}^{\sigma_{\Gamma}} for each stability codnition
    and dim_tree_poly is its dimension.

    """
    dic_l=[]
    top_dim=[] # record those with top dim poly
    degen=[] #record wsc with degen poly.
    
    for term,wsc in enumerate(data):
        
        #Use start to skip to the case you want to study.
        if term<start-1:
            continue
        print("Start=",start,"----",term,"----terminator:",terminator)
        
        #for Tree_poly
        assignments=wsc[2] #[(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0])]
        tree_poly=get_intersection_tree_poly(G,assignments)
        dim_tree_poly=dim(tree_poly)
        
        print("dim_tree_poly:",dim_tree_poly)
        dic={"Dimension of tree-poly":dim_tree_poly}
        dic_l.append(dic)
        
        if dim_tree_poly==size:
            top_dim.append(wsc)
        else:
            print(f"{term}: WSC not phi: \n",wsc)
            degen.append(wsc)
            
    #Store those wsc that fail and those that are top dim poly.
    wsc_ex=data[0] #example wsc to get assignment to get trees in order.
    ass_ex=wsc_ex[2] # [(T,ass),...]
    ordering_trees=[pair[0] for pair in ass_ex]
    
    #We store the data
    pickle_subset_true(graphname,top_dim)
    store_data_top_dim(G,top_dim,ordering_trees,graphname)
    pickle_subset_false(graphname,degen)
    store_data_degen(G,degen,ordering_trees,graphname)
    
    return dic_l

The following function returns a summary the results of analyse\_graphname and analyse\_graphname_FV.

In [ ]:
def analyse_table(dic_l):
    
    """
    Objective: Give a summary of results
    Input: list of dic={"Dimension of tree-poly":dim(tree_poly) }  
    Return:Summary text of results
    """
    
    num_wsc=[]#number of wsc
    dim_value_counts=[]

    for dic in dic_l:
    
        num_wsc.append(dic["Dimension of tree-poly"])
        dim_value_counts.append(dic["Dimension of tree-poly"])

    dim_value_counts_dict=dict(Counter(dim_value_counts)) # dictionary of number of terms of a given dim    
    print(textwrap.dedent(f'''\
        Total number of stability conditions {len(num_wsc)}
        Number of polytopes of a given (dimension:number of at dim): {dim_value_counts_dict}
    '''))
    return 

## Loading and Storing data <a name="s13"></a>

The following functions load the pkl file stored in "examples/{graphname}/{graphname}.pkl" depending on the graphname choosen.

In [ ]:
def unpickle(graphname):
    # Unpickle the list
    filename=f"examples/{graphname}/{graphname}.pkl"
    with open(filename, 'rb') as f:
        unpickled_list = pickle.load(f)
    return unpickled_list

The following function stores the dictionry output of analyse\_graphname. Which states how many stability conditions are of which dimension.

In [ ]:
def pickle_dic_l(graphname,dic_l):
    # Pickle the list
    filename=f"examples/{graphname}/phi_analysis/dic_l.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(dic_l, f)
    return

In the case of the graph $F$ the following functions store the pkl files of the output of the function analyse\_graphname\_FV. The first stores those with polytope which has dimension $|V(\Gamma) |-1$ the second those which do not.

In [ ]:
def pickle_subset_true(graphname,l):
    # Pickle the list
    filename=f"{graphname}_top_dim.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(l, f)
    f.close()  # Close the file after writing

    return

def pickle_subset_false(graphname,l):
    # Pickle the list
    filename=f"{graphname}_degen.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(l, f)
    f.close()  # Close the file after writing

    return

The following functions save a text description of the output of analyse\_graphname. In particular a description of each stability condition of either those with a polytope which has dimension $|V(\Gamma) |-1, or those that do not.

In [ ]:
#Storing files 
def output_to_file(G,total_data,ordering_trees):#in text files
    """
    Objective: GEt text file descibing stability conditions
    Input: total_data is a set of stability conditions.
    ordering_trees is a list of spanning trees described by edges.
    Returns: text file
    
    Return to text file:
    - add list of subgraphs under trees.
    - record list of cycles for subg, after lbm. 
    """

    print(f"Information: Here are all stability conditions for the graph G")
    print(f"Graph: {G.edges()} \n")
    
    print(f"There are {len(total_data)} stability conditions for G. \n")
    
    print(f"The spanning trees of G are ordered as follows:")
    for index,tree in enumerate(ordering_trees):
        print(f"Tree {index+1}: {tree.edges()}" )
        
    print(f"\n The spanning subgraphs of G of the for T cup e are ordered as follows:")
    
    """
    Fix order of subg printed:
    
    Here get_subgraphs_gm1_Edgesmissing does not reflect the ordering of subg in subg_cycle_l=tup[3]
    subgraphs= get_subgraphs_gm1_Edgesmissing(G) #Get #\tilde{G} the subgraphs Tcup e.-D

    #Assuming subg_cycle_l for each element of total_data is ordered the same subg.
    """

    tup_example=total_data[0]
    subg_cycle_l=tup_example[3]
    subg_ll=[pair[0] for pair in subg_cycle_l]

    for index,subg in enumerate(subg_ll):
        print(f"Subgraph {index+1}: {subg.edges()}" )
    

    print("\n We will now list all stabilty conditions. The data will be presented in three lines:")
    print("   1. The assignment datum on trees the order of which will be given by the Tree order,")
    print("   2. The union of sets of the form assignment datum of T + the set of break divisors for T, " )          
    print("   3. The cycle used with th subgraph to obtain the assignments on trees (ordered by subgraphs above).\n")

    for tup in total_data:
        
        print(tup[0]) #assignment datum
        
        #Put sigma to list format
        print(tup[1].tolist()) #lbm
        
        subg_cycle_l=tup[3]
        l_cycles=[(x[1],x[2].edges()) for x in subg_cycle_l] #records cycle and tree for sub.
        print(l_cycles)
        
        print("\n")
    
    return

In [ ]:
def store_data_top_dim(G,total_data,ordering_trees,graphname="test"):
    name=f"{graphname}_top_dim.txt"
    print(name)
    with open(name, "w") as f:
        sys.stdout = f
#         print("before output")
        output_to_file(G,total_data,ordering_trees)
#         print("after output")
        
        sys.stdout = sys.__stdout__
#         print("here closes??")
    f.close()  # Close the file after writing
#     print("does it close????")
    return

def store_data_degen(G,total_data,ordering_trees,graphname="test"):
    name=f"{graphname}_degen.txt"
    print(name)
    with open(name, "w") as f:
        sys.stdout = f

        output_to_file(G,total_data,ordering_trees)

        sys.stdout = sys.__stdout__
    f.close()  # Close the file after writing

    return

# Example <a name="s3"></a>

To help us understand the above functions let us fix $\Gamma$ to be the $3$-necklace graph.

In [ ]:
# graphname="G3"
G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
spanning_trees=[tree for tree in G.spanning_trees()]
T=spanning_trees[0]
# plot(G, figsize=3)

We see we can obtain all pair of elementary induced graphs

In [ ]:
get_pairs_con_subg(G)

For both unique stability conditions for the $3$-necklace graph we see that $P_{ST}^{\sigma_{\Gamma}}$ has dimension $2$.

In [ ]:
#Trivial I3 example
assignments=[(spanning_trees[0],[0,0,0]),(spanning_trees[1],[0,0,0]),(spanning_trees[2],[0,0,0])]
inter_triv=get_intersection_tree_poly(G,assignments)
print(dim(inter_triv))

In [ ]:
#Non-trivial I3 example
assignments=[(spanning_trees[0],[0,0,0]),(spanning_trees[1],[0,1,-1]),(spanning_trees[2],[1,0,-1])]
inter_nontriv=get_intersection_tree_poly(G,assignments)
print(dim(inter_nontriv))

# Cases <a name="s2"></a>

We now determine for the graphs of Section $1.3$ of the thesis whether each graph $\Gamma$, the convex region $P_{ST}^{\sigma_{\Gamma}}$ is empty or not for all $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ (obtained via 4\_assignments.ipynb and 5_assignments\_11k3graphs.ipynb),

## $n$-necklace graphs $n \le 5$ <a name="s21"></a>

Even though we know by Section $3.1$ that for every genus $1$ graph, every stability conditions is given by a numerical polatisation, we check the first few cases to ensure the methods are correct.

### $3$-necklace

In [ ]:
%%time
graphname="G3"
G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)

data=unpickle(graphname) #Size=2

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $4$-necklace

In [ ]:
%%time
graphname="G4"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0")], multiedges=True)

data=unpickle(graphname) #Size=6

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $5$-necklace

In [ ]:
%%time
graphname="G5"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0")], multiedges=True)

data=unpickle(graphname) #Size=24

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

## $\Gamma:=\Gamma_{1 1 k_3}$ for $2 \le k_3 \le 6$  <a name="s22"></a>

We now consider the dimension of $P_{ST}^{\sigma_{\Gamma}}$ for $ \sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ for graphs  
$\Gamma:=\Gamma_{k_1 k_2 k_3}$ with $(k_1,k_2,k_3)=\{(1,1,2),(1,1,3),(1,1,4),(1,1,5),(1,1,6)\}$. 

### $(k_1,k_2,k_3)=(1,1,2)$

In [ ]:
%%time
graphname="G3M1" 
G=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12")], multiedges=True)

data=unpickle(graphname) #Size=2

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,1,3)$

In [ ]:
%%time
graphname="G4M1" 
G=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","0","e4"),("0","1","e12")], multiedges=True)

data=unpickle(graphname) #Size=6

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,1,4)$

In [ ]:
%%time
graphname="G5M1" 
G=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","4","e4"),("4","0","e5"),("0","1","e12")], multiedges=True)

data=unpickle(graphname) #Size=24

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,1,5)$

In [ ]:
%%time
graphname="G6M1" 
G=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","4","e4"),("4","5","e5"),("5","0","e6"),("0","1","e12")], multiedges=True)

data=unpickle(graphname) #Size=120

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,1,6)$

In [ ]:
%%time
graphname="G7M1" 
G=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","4","e4"),("4","5","e5"),("5","6","e6"),("6","0","e7"),("0","1","e12")], multiedges=True)

data=unpickle(graphname) #Size=720

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

## $\Gamma:=\Gamma_{1 k_2 k_3}$ for $k_2\ge 2$  <a name="s23"></a>

We now consider the dimension of $P_{ST}^{\sigma_{\Gamma}}$ for $ \sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ for graphs   $\Gamma:=\Gamma_{k_1 k_2 k_3}$ with $(k_1,k_2,k_3)=\{(1,2,2),(1,2,3),(1,2,4),(1,3,3),(1,2,5),(1,3,4)\}$. 

### $(k_1,k_2,k_3)=(1,2,2)$

In [ ]:
%%time
graphname="G4M2" 
G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","2")], multiedges=True)

data=unpickle(graphname) #Size=6

dic_l=analyse_graphname(G,data)

In [ ]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,2,3)$

In [ ]:
%%time
graphname="G5M2"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2")], multiedges=True)

data=unpickle(graphname) #Size=24

dic_l=analyse_graphname(G,data)

In [ ]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,2,4)$

In [ ]:
%%time
graphname="G6M2"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","2")], multiedges=True)

data=unpickle(graphname) #Size=120

dic_l=analyse_graphname(G,data)

In [ ]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,3,3)$

In [ ]:
%%time
graphname="G6M3"  
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","3")], multiedges=True)

data=unpickle(graphname) #Size=120

dic_l=analyse_graphname(G,data)

In [ ]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,2,5)$

In [ ]:
%%time
graphname="G7M02"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","2")], multiedges=True)

data=unpickle(graphname) 

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $(k_1,k_2,k_3)=(1,3,4)$

In [ ]:
%%time
graphname="G7M03"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","3")], multiedges=True)

data=unpickle(graphname) 

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

## $\Gamma:=\Gamma_{k_1 k_2 k_3}$ for $k_1\ge 2$  <a name="s24"></a>

We now consider the dimension of $P_{ST}^{\sigma_{\Gamma}}$ for $ \sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ for graphs   $\Gamma:=\Gamma_{k_1 k_2 k_3}$ with  $(k_1,k_2,k_3)=\{(2,2,2),(2,2,3),(2,2,4),(2,3,3)\}$. 

### $(k_1,k_2,k_3)=(2,2,2)$


In [ ]:
%%time
graphname="V_222"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","4"),("4","2")], multiedges=True)

data=unpickle(graphname) 

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)


### $(k_1,k_2,k_3)=(2,2,3)$


In [ ]:
%%time
graphname="V_223"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","5"),("5","2")], multiedges=True)

data=unpickle(graphname) 

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)


### $(k_1,k_2,k_3)=(2,2,4)$


In [ ]:
%%time
graphname="V_224"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","6"),("6","2")], multiedges=True)

data=unpickle(graphname) 

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)


### $(k_1,k_2,k_3)=(2,3,3)$

In [ ]:
%%time
graphname="V_233"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","6"),("6","3")], multiedges=True)

data=unpickle(graphname) 

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

## $\Gamma \in \{ G_1,G_2,G_3,F\}$ <a name="s25"></a>

We now consider the dimension of $P_{ST}^{\sigma_{\Gamma}}$ for $ \sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ for graphs   $\Gamma\in \{G_1,G_2,G_3,F\}$.

### $G_1$

In [ ]:
%%time
graphname="G4M02M13"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","2"),("1","3")], multiedges=True)

data=unpickle(graphname) #Size=10

dic_l=analyse_graphname(G,data)

In [ ]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $G_2$

In [ ]:
%%time
graphname="G5M02M03"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3")], multiedges=True)

data=unpickle(graphname) #Size=24

dic_l=analyse_graphname(G,data)

In [ ]:
#Store list of dictionaries
pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $G_3$

In [ ]:
%%time
graphname="G5M02M03M14"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3"),("1","4")], multiedges=True)

data=unpickle(graphname) #Size=82

dic_l=analyse_graphname(G,data)

In [ ]:
#Store list of dictionaries
pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l)

### $F$

Unlike the other cases when we run analyse\_graphname we have $\sigma_{\Gamma}$ such that $P_{ST}^{\sigma_{\Gamma}}$ has dimension less than $|V(\Gamma)|-1=5$ which by the discussion in the introduction implies that $P_{ST}^{\sigma_{\Gamma}} = \emptyset$. We have therefore modified analyse\_graphname to analyse_graphname\_FV to parse the cases of $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ with $P_{ST}^{\sigma_{\Gamma}} \ne \emptyset$ and those with $P_{ST}^{\sigma_{\Gamma}} = \emptyset$.

In [ ]:
%%time
graphname="FV_G6M3M14"  
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","3"),("1","4")], multiedges=True)

data=unpickle(graphname) #Size=504

In [ ]:
dim_size=5 #number of vertices minus 1.
dic_l=analyse_graphname_FV(dim_size,graphname,G,data)

# Leftovers

In [ ]:
# def unpickle_dic_l(graphname):
#     # Unpickle the list
#     filename=f"examples/{graphname}/phi_analysis/dic_l.pkl"
#     with open(filename, 'rb') as f:
#         unpickled_list = pickle.load(f)
#     return unpickled_list